In [1]:
import geopandas as gpd
import os
import shapely
import subprocess
from lib.parse import *
from lib.funcs import *
import fiona


In [2]:
# Enable fiona driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

# Read file
comercial_df = gpd.read_file('./data/LOTES/COMERCIAL/KML/Lts_Geral_Comercial.kml', driver='KML')

# Drop Z dimension of polygons that occurs often in kml 
comercial_df.geometry = comercial_df.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y, z: (x, y), polygon))

In [3]:
path = './data/LOTES/SERVIÇOS/KML/'

servicos_df = gpd.GeoDataFrame()
for filename in os.listdir(path):
    df = gpd.read_file(path+filename)
    df.geometry = df.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y, z: (x, y), polygon))
    servicos_df = servicos_df.append(df, ignore_index=True)

In [4]:
comercial_df = comercial_df.join(comercial_df.apply(parse_html, axis=1))
comercial_df = comercial_df.drop('Description', 1)

/home/fc-anjos/professional/joao_pessoa_new/lib/parse.py:9: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 9 of the file /home/fc-anjos/professional/joao_pessoa_new/lib/parse.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(html_str)


In [ ]:
servicos_df = servicos_df.join(servicos_df.apply(parse_html, axis=1))
servicos_df = servicos_df.drop('Description', 1)

In [ ]:
servicos_df.columns

In [ ]:
comercial_df = comercial_df[['Grupo', 'Atividade', 'Nume_Ativ', 'Segmento', 'Situacao', 'geometry', 'Area_Terre', 'Area_Total', 'Area_Predi']]
servicos_df = servicos_df[['Grupo', 'Atividade', 'Nume_Ativ', 'Segmento', 'Situacao', 'geometry', 'Area_Terre', 'Area_Total', 'Area_Predi']]

In [ ]:
def to_float(v):
    v = v.replace(',', '.')
    v = float(v)
    return v

to_convert = ['Area_Terre', 'Area_Predi', 'Area_Total']
dfs = [comercial_df, servicos_df]

for column in to_convert:
    for df in dfs:
        df[column] = df[column].apply(to_float)

In [ ]:
for column in to_convert:
    for df in dfs:
        indexed = df[df[column] == 0 ].index
        df.drop(indexed, inplace=True)

In [ ]:
comercial_df.to_file("./data/LOTES/COMERCIAL/SHP/lotes.shp")
servicos_df.to_file("./data/LOTES/SERVIÇOS/SHP/lotes.shp")